In [2]:
import pandas as pd
import numpy as np
import obspy
from obspy.core import Stream, Trace
from obspy.signal.trigger import classic_sta_lta, trigger_onset
import os
import datetime
import math # Necesitamos importar math para ceil

# --- 1. Entrada de parámetros del usuario (mejoras en la interacción) ---
print("--- Configuración de Parámetros para Detección Sísmica ---")

while True:
    file_path = input("Por favor, introduce la ruta COMPLETA de tu archivo TXT de datos sísmicos (ej. C:\\datos\\sismo.txt): ")
    # Normalizar la ruta para manejar barras invertidas/normales y asegurar que funcione en diferentes OS
    file_path = os.path.normpath(file_path).strip('"') # Eliminar comillas si el usuario las pega
    if os.path.exists(file_path):
        print(f"Ruta de archivo '{file_path}' encontrada. Procediendo...")
        break
    else:
        print(f"Error: La ruta '{file_path}' no existe o no se puede acceder.")
        print("Asegúrate de escribir la ruta correctamente y de que el archivo exista.")

# Obtener el directorio del archivo de entrada para la salida
output_directory = os.path.dirname(file_path)
output_event_data_filename = os.path.join(output_directory, "datos_eventos_detectados.csv")

while True:
    try:
        sampling_rate_str = input("Introduce la frecuencia de muestreo de tus datos en Hz (ej. 40.0): ")
        sampling_rate = float(sampling_rate_str)
        if sampling_rate <= 0:
            print("La frecuencia de muestreo debe ser un número positivo (mayor que 0).")
        else:
            break
    except ValueError:
        print("Entrada inválida. Por favor, introduce un número válido para la frecuencia de muestreo (ej. 40.0).")

# --- Solicitar fecha y hora de inicio ---
while True:
    start_date_str = input("Introduce la fecha de inicio de la muestra (YYYY-MM-DD, ej. 2023-01-15): ")
    start_time_str = input("Introduce la hora de inicio de la muestra (HH:MM:SS, ej. 10:30:00): ")
    try:
        start_datetime = datetime.datetime.strptime(f"{start_date_str} {start_time_str}", "%Y-%m-%d %H:%M:%S")
        print(f"Fecha y hora de inicio configuradas: {start_datetime}")
        break
    except ValueError:
        print("Formato de fecha u hora inválido. Por favor, usa YYYY-MM-DD y HH:MM:SS.")

while True:
    try:
        freq_min_bandpass_str = input("Introduce la frecuencia INFERIOR del filtro de paso de banda en Hz (ej. 1.0): ")
        freq_min_bandpass = float(freq_min_bandpass_str)
        if freq_min_bandpass < 0:
            print("La frecuencia mínima del filtro no puede ser negativa.")
        else:
            break
    except ValueError:
        print("Entrada inválida. Por favor, introduce un número para la frecuencia mínima del filtro.")

while True:
    try:
        freq_max_bandpass_str = input("Introduce la frecuencia SUPERIOR del filtro de paso de banda en Hz (ej. 5.0): ")
        freq_max_bandpass = float(freq_max_bandpass_str)
        if freq_max_bandpass <= freq_min_bandpass:
            print("La frecuencia superior debe ser mayor que la frecuencia inferior.")
        elif freq_max_bandpass >= (sampling_rate / 2):
            print(f"Advertencia: La frecuencia superior ({freq_max_bandpass} Hz) es igual o mayor que la frecuencia de Nyquist ({sampling_rate / 2} Hz).")
            print("Esto puede llevar a resultados inesperados (aliasing) o un filtrado ineficaz.")
            if input("¿Deseas continuar a pesar de esta advertencia? (s/n): ").lower() != 's':
                continue
            break # El usuario decidió continuar a pesar de la advertencia
        else:
            break
    except ValueError:
        print("Entrada inválida. Por favor, introduce un número para la frecuencia superior del filtro.")

# --- Parámetro para unificación de componentes (siempre True por diseño del script) ---
unify_components = True # Las componentes E, N, Z siempre se unificarán en 'Magnitude'.
print(f"Componentes unificadas: {unify_components} (fijo en este script a la magnitud vectorial).")

# --- Umbral de magnitud para filtrar (aplicado a la señal unificada y filtrada) ---
while True:
    try:
        magnitude_threshold_str = input("Introduce el umbral de amplitud para la señal unificada y filtrada (ej. 0.1). Las amplitudes menores serán descartadas: ")
        magnitude_threshold = float(magnitude_threshold_str)
        if magnitude_threshold < 0:
            print("El umbral de amplitud no puede ser negativo.")
        else:
            break
    except ValueError:
        print("Entrada inválida. Por favor, introduce un número para el umbral de amplitud.")

# --- Constante de calibración para la magnitud instrumental simplificada (Ml_Calculada_Simplificada) ---
while True:
    try:
        ml_calibration_constant_str = input("Introduce la constante de calibración para la magnitud instrumental simplificada (ej. 2.0). ESTE VALOR REQUIERE CALIBRACIÓN EMPÍRICA: ")
        ml_calibration_constant = float(ml_calibration_constant_str)
        # Una constante de calibración Ml puede ser positiva o negativa dependiendo de la región y sismógrafo.
        # No ponemos restricción de positivo aquí.
        break
    except ValueError:
        print("Entrada inválida. Por favor, introduce un número para la constante de calibración Ml.")

# --- Duración mínima del evento en segundos ---
while True:
    try:
        min_event_duration_str = input("Introduce la duración mínima de un evento (en segundos) para ser registrado (ej. 0.5): ")
        min_event_duration = float(min_event_duration_str)
        if min_event_duration <= 0:
            print("La duración mínima del evento debe ser un número positivo (mayor que 0).")
        else:
            break
    except ValueError:
        print("Entrada inválida. Por favor, introduce un número para la duración mínima del evento.")

# --- Parámetros fijos para STA/LTA y filtro ---
# Estos parámetros son internos y se pueden ajustar directamente en el código si es necesario.
corners = 4 # Número de esquinas para el filtro (mayor número = filtro más empinado)
stalta_short = 1.0 # Ventana STA en segundos
stalta_long = 10.0 # Ventana LTA en segundos
thresh_on = 2.5 # Umbral para activar la detección
thresh_off = 0.8 # Umbral para desactivar la detección


# --- Cálculo dinámico del tamaño del chunk y validación ---
print("\n--- Calculando el tamaño de los bloques para dividir el archivo... ---")
try:
    with open(file_path, 'r') as f:
        # Contar líneas, excluyendo posibles encabezados si tu TXT los tuviera.
        # Asumimos que no hay encabezado por tu uso de names=['E', 'N', 'Z'] en pd.read_csv.
        total_lines = sum(1 for line in f)
    
    if total_lines == 0:
        raise ValueError("El archivo de datos está vacío.")

    # Número de partes deseado. Se podría hacer configurable también.
    num_parts = 50 
    chunk_size = int(math.ceil(total_lines / num_parts))
    
    if chunk_size == 0: # Si el archivo es muy pequeño
        chunk_size = total_lines 

    print(f"El archivo tiene {total_lines} registros.")
    print(f"Se dividirá en aproximadamente {int(np.ceil(total_lines / chunk_size))} bloques de {chunk_size} registros cada uno.")

    # --- ADVERTENCIA CRÍTICA SOBRE EL TAMAÑO DEL CHUNK Y STA/LTA ---
    min_samples_for_stalta_LTA_window = int(stalta_long * sampling_rate)
    min_recommended_chunk_for_stalta = int(min_samples_for_stalta_LTA_window * 2.5) # Recomendación: al menos 2.5 veces la ventana LTA

    if chunk_size < min_recommended_chunk_for_stalta:
        print(f"\n¡¡¡ADVERTENCIA CRÍTICA!!!")
        print(f"El tamaño de cada bloque ({chunk_size} muestras) es MUY PEQUEÑO para la ventana larga de STA/LTA ({stalta_long}s = {min_samples_for_stalta_LTA_window} muestras).")
        print(f"Esto causará que el algoritmo STA/LTA NO SE EJECUTE CORRECTAMENTE o NO DETECTE EVENTOS en la mayoría de los bloques, especialmente cerca de los bordes.")
        print(f"Se recomienda que el tamaño del bloque sea al menos {min_recommended_chunk_for_stalta} muestras para que STA/LTA funcione de manera fiable.")
        print(f"Considera dividir el archivo en MENOS bloques (ej. {int(np.ceil(total_lines / min_recommended_chunk_for_stalta))} en lugar de {num_parts} para un chunk_size de {min_recommended_chunk_for_stalta}) o ajustar tus ventanas STA/LTA (stalta_long).")
        
        while True:
            confirm = input("¿Deseas continuar a pesar de esta advertencia? (s/n): ").lower()
            if confirm == 's':
                break
            elif confirm == 'n':
                print("Procesamiento cancelado por el usuario.")
                exit() # Sale del script
            else:
                print("Respuesta inválida. Por favor, introduce 's' o 'n'.")

except Exception as e:
    print(f"Error al calcular el tamaño total de líneas o el tamaño del chunk: {e}")
    print("Asegúrate de que la ruta del archivo sea correcta y que el archivo no esté vacío.")
    exit()


print(f"\n--- Configuración Final del Procesamiento ---")
print(f"  Archivo TXT de entrada: {file_path}")
print(f"  Directorio de salida: {output_directory}")
print(f"  Frecuencia de muestreo: {sampling_rate} Hz")
print(f"  Fecha y hora de inicio de los datos: {start_datetime}")
print(f"  Filtro de paso de banda: {freq_min_bandpass}-{freq_max_bandpass} Hz (orden: {corners})")
print(f"  Unificación de componentes (E,N,Z a Magnitud): Sí")
print(f"  Umbral de amplitud para filtrar ('Magnitude_Filtered'): {magnitude_threshold}")
print(f"  Constante de calibración Ml (simplificada): {ml_calibration_constant}") 
print(f"  Duración mínima de evento para registro: {min_event_duration}s")
print(f"  Tamaño de bloque para procesamiento: {chunk_size} muestras")
print(f"  Ventana STA (corta): {stalta_short}s")
print(f"  Ventana LTA (larga): {stalta_long}s")
print(f"  Umbral STA/LTA ON: {thresh_on}")
print(f"  Umbral STA/LTA OFF: {thresh_off}")

print("\n--- Iniciando el procesamiento del archivo... ---")

# --- Procesamiento por Chunks ---
try:
    start_time_current_chunk = start_datetime
    
    all_processed_chunks_for_final_df = [] # Lista para almacenar DataFrames de chunks procesados
    detected_event_summary_list = [] # Lista para almacenar los detalles de los eventos detectados

    # Lee el archivo por chunks usando pandas
    for i, chunk_df in enumerate(pd.read_csv(file_path, sep='\s+', header=None, names=['E', 'N', 'Z'], chunksize=chunk_size)):
        print(f"\nProcesando bloque {i+1} de aproximadamente {int(np.ceil(total_lines / chunk_size))}...")
        
        # --- Calcular 'timestamp' para el chunk actual ---
        # El número de muestras en el chunk es len(chunk_df)
        chunk_df['timestamp'] = [start_time_current_chunk + datetime.timedelta(seconds=j / sampling_rate) 
                                 for j in range(len(chunk_df))]
        
        # --- Actualizar el tiempo de inicio para el siguiente chunk ---
        # Esto asegura la continuidad temporal entre bloques
        if not chunk_df.empty: # Asegurarse de que el chunk no esté vacío
            start_time_current_chunk = chunk_df['timestamp'].iloc[-1] + datetime.timedelta(seconds=1 / sampling_rate)
        else:
            # Si el chunk está vacío después de la lectura, el siguiente chunk comenzará en el mismo tiempo
            # o en el tiempo anterior actualizado si hubiera habido datos. Esto previene un salto si un chunk está vacío.
            print(f"Advertencia: El chunk {i+1} no contenía datos válidos en la lectura inicial. La continuidad temporal podría verse afectada.")

        # --- Limpieza de datos no numéricos y unificación de componentes ---
        for col in ['E', 'N', 'Z']:
            chunk_df[col] = pd.to_numeric(chunk_df[col], errors='coerce') # Convierte a numérico, errores a NaN
        chunk_df.dropna(subset=['E', 'N', 'Z'], inplace=True) # Elimina filas con NaN en E, N o Z

        if chunk_df.empty:
            print(f"Advertencia: El bloque {i+1} quedó vacío después de la limpieza de datos no numéricos. Saltando a la siguiente...")
            continue # Salta al siguiente chunk si este está vacío

        # Calcula la magnitud vectorial (norma L2)
        chunk_df['Magnitude'] = np.sqrt(chunk_df['E']**2 + chunk_df['N']**2 + chunk_df['Z']**2)
        data_to_filter = chunk_df['Magnitude'].values
        
        # --- Aplicar filtro pasa-banda usando Obspy Trace ---
        trace = Trace(data=data_to_filter)
        trace.stats.sampling_rate = sampling_rate
        # Se detrae y se aplica filtro
        trace.detrend("constant") # Eliminar media
        trace.detrend("linear")   # Eliminar tendencia lineal
        trace.filter('bandpass', freqmin=freq_min_bandpass, freqmax=freq_max_bandpass, corners=corners, zerophase=True)
        
        chunk_df['Magnitude_Filtered'] = trace.data # Asignar los datos filtrados de vuelta al DataFrame

        # --- Filtrar el DataFrame por el umbral de amplitud (Magnitude_Filtered) ---
        # Ahora, solo las filas que superan el umbral de amplitud serán consideradas para STA/LTA
        current_chunk_filtered_by_amplitude = chunk_df[chunk_df['Magnitude_Filtered'] > magnitude_threshold].copy()
        
        if current_chunk_filtered_by_amplitude.empty:
            print(f"Advertencia: El bloque {i+1} no tiene datos que superen el umbral de amplitud ({magnitude_threshold}). Saltando detección STA/LTA para este bloque.")
            # Aunque no haya datos por encima del umbral para STA/LTA, aún queremos almacenar el chunk original completo
            # con las columnas Magnitude y Magnitude_Filtered para el DataFrame final si no aplicamos el umbral para el dataframe final.
            # Aquí, lo que se añadió a `all_processed_chunks_for_final_df` es el `current_chunk_processed` que es `chunk_df` original.
            # Si se desea que el DataFrame final SÓLO contenga lo que superó el umbral, se usará `current_chunk_filtered_by_amplitude`
            # He decidido mantener la adición de `chunk_df` completo para que el usuario pueda ver la señal completa filtrada.
            all_processed_chunks_for_final_df.append(chunk_df) # Añadir el chunk original con las columnas Magnitude y Magnitude_Filtered
            continue # Saltar a la siguiente iteración del loop, no hay datos para STA/LTA en este chunk

        # Añadir el chunk procesado (con todas sus filas originales y las columnas añadidas)
        # a la lista para el DataFrame final, si no se filtró todo.
        all_processed_chunks_for_final_df.append(chunk_df)


        # --- Aplicar STA/LTA en la señal filtrada por amplitud ---
        data_for_stalta = current_chunk_filtered_by_amplitude['Magnitude_Filtered'].values
        
        # Validar si hay suficientes puntos para STA/LTA en el chunk actual
        npts_long = int(stalta_long * sampling_rate) # Número de muestras para la ventana larga
        npts_short = int(stalta_short * sampling_rate) # Número de muestras para la ventana corta

        if len(data_for_stalta) < max(npts_long, npts_short) + 1: # Se necesita al menos la ventana más grande + 1 para el cálculo
            print(f"Advertencia: El bloque {i+1} después del filtrado por amplitud ({len(data_for_stalta)} muestras) es demasiado corto para STA/LTA (se necesitan al menos {max(npts_long, npts_short) + 1} muestras). Saltando detección para este bloque.")
            continue # Salta a la siguiente iteración, no se puede aplicar STA/LTA

        cft = classic_sta_lta(data_for_stalta, npts_short, npts_long)
        on_off = trigger_onset(cft, thresh_on, thresh_off)
        
        if on_off.size > 0:
            # Los índices 'onset' y 'offset' se refieren al array `data_for_stalta`
            # Necesitamos mapearlos de vuelta al DataFrame `current_chunk_filtered_by_amplitude`
            
            for onset_idx_in_stalta_data, offset_idx_in_stalta_data in on_off:
                # Obtener los timestamps correspondientes del DataFrame `current_chunk_filtered_by_amplitude`
                # que es el subconjunto de `chunk_df` que pasó el `magnitude_threshold`
                
                # Asegurarse de que los índices estén dentro de los límites
                if onset_idx_in_stalta_data < len(current_chunk_filtered_by_amplitude) and \
                   offset_idx_in_stalta_data < len(current_chunk_filtered_by_amplitude):
                    
                    start_time_event = current_chunk_filtered_by_amplitude['timestamp'].iloc[onset_idx_in_stalta_data]
                    end_time_event = current_chunk_filtered_by_amplitude['timestamp'].iloc[offset_idx_in_stalta_data]
                    event_duration = (end_time_event - start_time_event).total_seconds()
                    
                    if event_duration >= min_event_duration:
                        # Calcular la máxima amplitud del evento dentro del periodo detectado
                        # Acceder a los datos originales del chunk_df usando los timestamps
                        event_data_segment = chunk_df[
                            (chunk_df['timestamp'] >= start_time_event) &
                            (chunk_df['timestamp'] <= end_time_event)
                        ]['Magnitude_Filtered']
                        
                        if not event_data_segment.empty:
                            max_amplitude_in_event = event_data_segment.max()
                            
                            # Calcular la magnitud instrumental simplificada (Ml)
                            # Se añade un pequeño valor para evitar log10(0) si la amplitud es 0
                            ml_value = np.log10(max_amplitude_in_event + 1e-9) + ml_calibration_constant 

                            detected_event_summary_list.append({
                                'Start_Time': start_time_event,
                                'End_Time': end_time_event,
                                'Duration_s': event_duration,
                                'Max_Amplitude_Filtered': max_amplitude_in_event,
                                'Ml_Calculada_Simplificada': ml_value
                            })
                        else:
                            print(f"Advertencia: Segmento de datos vacío para el evento detectado en {start_time_event}.")
                else:
                    print(f"Advertencia: Índices de STA/LTA fuera de rango en bloque {i+1}. Posiblemente un evento en el borde.")

except FileNotFoundError:
    print(f"Error: El archivo '{file_path}' no fue encontrado. Verifica la ruta e inténtalo de nuevo.")
    exit()
except Exception as e:
    print(f"Ocurrió un error inesperado durante el procesamiento. Por favor, revisa tus datos o parámetros: {e}")
    exit()

# --- Consolidar y Exportar Resultados ---

final_processed_df_with_all_chunks = pd.DataFrame()
if all_processed_chunks_for_final_df:
    print("\n--- Concatenando todos los bloques procesados en un único DataFrame para la salida final... ---")
    final_processed_df_with_all_chunks = pd.concat(all_processed_chunks_for_final_df).reset_index(drop=True)
    print(f"DataFrame consolidado tiene {len(final_processed_df_with_all_chunks)} filas.")
else:
    print("\nNo se procesó ningún bloque de datos. No se generará un DataFrame final.")


if not final_processed_df_with_all_chunks.empty and detected_event_summary_list:
    print("\n--- Generando el archivo de salida con datos sísmicos dentro de los eventos detectados... ---")
    
    events_df_summary = pd.DataFrame(detected_event_summary_list)
    event_data_rows_for_final_output = []

    # Iterar a través de cada evento detectado para extraer los datos correspondientes
    for index, event in events_df_summary.iterrows():
        event_start = event['Start_Time']
        event_end = event['End_Time']
        event_duration = event['Duration_s']
        max_amplitude = event['Max_Amplitude_Filtered']
        ml_value = event['Ml_Calculada_Simplificada']

        # Seleccionar las filas del DataFrame consolidado que caen dentro del rango de tiempo de este evento
        data_in_event = final_processed_df_with_all_chunks[
            (final_processed_df_with_all_chunks['timestamp'] >= event_start) & 
            (final_processed_df_with_all_chunks['timestamp'] <= event_end)
        ].copy() 

        if not data_in_event.empty:
            # Añadir metadatos del evento a cada fila de datos de ese evento
            data_in_event['Fecha_Evento'] = event_start.strftime('%Y-%m-%d')
            data_in_event['Hora_Inicio_Evento'] = event_start.strftime('%H:%M:%S.%f')[:-3] # ms
            data_in_event['Hora_Fin_Evento'] = event_end.strftime('%H:%M:%S.%f')[:-3] # ms
            data_in_event['Duracion_Evento_s'] = event_duration
            data_in_event['Max_Amplitud_Filtrada'] = max_amplitude
            data_in_event['Ml_Calculada_Simplificada'] = ml_value

            # Dividir el timestamp original en 'Fecha_Lectura' y 'Hora_Lectura'
            data_in_event['Fecha_Lectura'] = data_in_event['timestamp'].dt.strftime('%Y-%m-%d')
            data_in_event['Hora_Lectura'] = data_in_event['timestamp'].dt.strftime('%H:%M:%S.%f')[:-3] # ms

            # Definir las columnas deseadas y su orden para el archivo final
            cols_to_keep = [
                'Fecha_Evento', 'Hora_Inicio_Evento', 'Hora_Fin_Evento', 'Duracion_Evento_s',
                'Max_Amplitud_Filtrada', 'Ml_Calculada_Simplificada', 
                'Fecha_Lectura', 'Hora_Lectura', 
                'E', 'N', 'Z', 'Magnitude', 'Magnitude_Filtered'
            ]
            
            # Seleccionar y reordenar las columnas deseadas
            data_in_event = data_in_event[cols_to_keep]
            event_data_rows_for_final_output.append(data_in_event)

    if event_data_rows_for_final_output:
        final_output_df = pd.concat(event_data_rows_for_final_output).reset_index(drop=True)
        final_output_df.to_csv(output_event_data_filename, index=False)
        print(f"\nEl archivo de salida con los datos sísmicos de los eventos detectados se ha guardado en:\n{output_event_data_filename}")
        print(f"Total de registros de eventos detectados exportados: {len(final_output_df)}.")
    else:
        print("\nNo se encontraron datos sísmicos dentro de los eventos detectados que cumplan todos los criterios.")
else:
    print("\nNo se pudo generar el archivo de datos de eventos filtrados (no se detectaron eventos o el DataFrame consolidado está vacío).")

print("\n--- Procesamiento completado ---")

<>:200: SyntaxWarning: invalid escape sequence '\s'
<>:200: SyntaxWarning: invalid escape sequence '\s'
C:\Users\USUARIO\AppData\Local\Temp\ipykernel_1656\3907847984.py:200: SyntaxWarning: invalid escape sequence '\s'
  for i, chunk_df in enumerate(pd.read_csv(file_path, sep='\s+', header=None, names=['E', 'N', 'Z'], chunksize=chunk_size)):


--- Configuración de Parámetros para Detección Sísmica ---


Por favor, introduce la ruta COMPLETA de tu archivo TXT de datos sísmicos (ej. C:\datos\sismo.txt):  F:\gustavo\Desktop\PracticasProfecionalizantes_1_2025\TRVA.201502.E.N.Z.txt


Ruta de archivo 'F:\gustavo\Desktop\PracticasProfecionalizantes_1_2025\TRVA.201502.E.N.Z.txt' encontrada. Procediendo...


Introduce la frecuencia de muestreo de tus datos en Hz (ej. 40.0):  40
Introduce la fecha de inicio de la muestra (YYYY-MM-DD, ej. 2023-01-15):  2015-02-01
Introduce la hora de inicio de la muestra (HH:MM:SS, ej. 10:30:00):  00:00:00


Fecha y hora de inicio configuradas: 2015-02-01 00:00:00


Introduce la frecuencia INFERIOR del filtro de paso de banda en Hz (ej. 1.0):  1.0
Introduce la frecuencia SUPERIOR del filtro de paso de banda en Hz (ej. 5.0):  5.0


Componentes unificadas: True (fijo en este script a la magnitud vectorial).


Introduce el umbral de amplitud para la señal unificada y filtrada (ej. 0.1). Las amplitudes menores serán descartadas:  2.5
Introduce la constante de calibración para la magnitud instrumental simplificada (ej. 2.0). ESTE VALOR REQUIERE CALIBRACIÓN EMPÍRICA:  2.0
Introduce la duración mínima de un evento (en segundos) para ser registrado (ej. 0.5):  10



--- Calculando el tamaño de los bloques para dividir el archivo... ---
El archivo tiene 96768200 registros.
Se dividirá en aproximadamente 50 bloques de 1935364 registros cada uno.

--- Configuración Final del Procesamiento ---
  Archivo TXT de entrada: F:\gustavo\Desktop\PracticasProfecionalizantes_1_2025\TRVA.201502.E.N.Z.txt
  Directorio de salida: F:\gustavo\Desktop\PracticasProfecionalizantes_1_2025
  Frecuencia de muestreo: 40.0 Hz
  Fecha y hora de inicio de los datos: 2015-02-01 00:00:00
  Filtro de paso de banda: 1.0-5.0 Hz (orden: 4)
  Unificación de componentes (E,N,Z a Magnitud): Sí
  Umbral de amplitud para filtrar ('Magnitude_Filtered'): 2.5
  Constante de calibración Ml (simplificada): 2.0
  Duración mínima de evento para registro: 10.0s
  Tamaño de bloque para procesamiento: 1935364 muestras
  Ventana STA (corta): 1.0s
  Ventana LTA (larga): 10.0s
  Umbral STA/LTA ON: 2.5
  Umbral STA/LTA OFF: 0.8

--- Iniciando el procesamiento del archivo... ---

Procesando bloque 1 